In [1]:
import pickle

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
with open('split_data.pkl','rb') as f:
    data = pickle.load(f)

x_train = data['x_train']
x_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']

In [3]:
# Standard Scaler 

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

### Autoencoder for Unsupervised Representation Learning

In [ ]:
early_stop_ae = keras.callbacks.EarlyStopping(
    monitor="val_loss",         
    patience=5,                 
    restore_best_weights=True   
)

In [5]:
input_layer = layers.Input(shape=(X_train_scaled.shape[1],))
encoded = layers.Dense(32, activation="relu")(input_layer)
encoded = layers.Dense(16, activation="relu")(encoded)
encoded_output = layers.Dense(8, activation="relu")(encoded)

decoded = layers.Dense(16, activation="relu")(encoded_output)
decoded = layers.Dense(32, activation="relu")(decoded)
decoded_output = layers.Dense(X_train_scaled.shape[1], activation="linear")(decoded)

autoencoder = keras.Model(inputs=input_layer, outputs=decoded_output)
encoder = keras.Model(inputs=input_layer, outputs=encoded_output)

In [6]:
autoencoder.compile(
  optimizer="adam",
  loss='mean_squared_error',
  metrics=['mean_absolute_error', 'mean_squared_error']
  )

In [ ]:

autoencoder.fit(
    X_train_scaled, X_train_scaled,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop_ae],
    verbose=1
)

Epoch 1/30
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.6368 - mean_absolute_error: 0.5073 - mean_squared_error: 0.6368 - val_loss: 0.3014 - val_mean_absolute_error: 0.3463 - val_mean_squared_error: 0.3014
Epoch 2/30
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.3069 - mean_absolute_error: 0.3450 - mean_squared_error: 0.3069 - val_loss: 0.2199 - val_mean_absolute_error: 0.3052 - val_mean_squared_error: 0.2199
Epoch 3/30
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.2258 - mean_absolute_error: 0.3086 - mean_squared_error: 0.2258 - val_loss: 0.1885 - val_mean_absolute_error: 0.2899 - val_mean_squared_error: 0.1885
Epoch 4/30
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1925 - mean_absolute_error: 0.2878 - mean_squared_error: 0.1925 - val_loss: 0.1737 - val_mean_absolute_error: 0.2779 - val_mean_squared_error: 0.1737
Epoch 5/30
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1686 - mean_absolute_error: 0.2667 - mean_squared_error: 0.1686 - val_loss: 0.16

### Generate Embeddings

In [9]:
X_train_embed = encoder.predict(X_train_scaled)
X_test_embed  = encoder.predict(X_test_scaled)

2850/2850 ━━━━━━━━━━━━━━━━━━━━ 1s 509us/step
713/713 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step


### Clustering on Embeddings

In [10]:
kmeans = KMeans(n_clusters=10, random_state=42)
train_clusters = kmeans.fit_predict(X_train_embed)
test_clusters  = kmeans.predict(X_test_embed)

In [11]:
# Add cluster as a new feature

X_train_final = np.concatenate([X_train_scaled, X_train_embed, train_clusters.reshape(-1,1)], axis=1)
X_test_final  = np.concatenate([X_test_scaled,  X_test_embed,  test_clusters.reshape(-1,1)], axis=1)

### Attention Layer

In [12]:
class AttentionLayer(layers.Layer):
    def call(self, inputs):
        weights = tf.nn.softmax(inputs, axis=1)
        return inputs * weights

### Deep Neural Network (DNN)

In [ ]:
# Early stopping
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",         
    patience=10,               
    restore_best_weights=True   
)

In [ ]:
model = keras.Sequential([
    layers.Input(shape=(X_train_final.shape[1],)),
    AttentionLayer(),
    layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="linear")  
])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ attention_layer                 │ (None, 28)             │             0 │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │         7,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,201 (200.00 KB)

 Trainable params: 51,201 (200.00 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=["mae", "mse"]
)

In [17]:
history = model.fit(
    X_train_final, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 432.2245 - mae: 16.3185 - mse: 432.2245 - val_loss: 343.3291 - val_mae: 14.3038 - val_mse: 343.3291
Epoch 2/100
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 321.8345 - mae: 13.8145 - mse: 321.8345 - val_loss: 317.4845 - val_mae: 13.6712 - val_mse: 317.4845
Epoch 3/100
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 306.8105 - mae: 13.5181 - mse: 306.8105 - val_loss: 318.5405 - val_mae: 13.7108 - val_mse: 318.5405
Epoch 4/100
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 302.5040 - mae: 13.2417 - mse: 302.5040 - val_loss: 298.5368 - val_mae: 13.2213 - val_mse: 298.5368
Epoch 5/100
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 297.3577 - mae: 13.1901 - mse: 297.3577 - val_loss: 306.0950 - val_mae: 13.3661 - val_mse: 306.0950
Epoch 6/100
2280/2280 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 293.4340 - mae: 13.0084 - mse: 293.4340 - val_loss: 294.8284 - val_mae: 13.0897 - val_mse: 294.8284
Epoch 7/100
2280/2280 

In [ ]:
loss, mae, mse = model.evaluate(X_test_final, y_test)
print("MAE:", mae, "  MSE:", mse)
model.save("deep_cluster_attention_model.keras")

print("Model saved successfully.")


713/713 ━━━━━━━━━━━━━━━━━━━━ 1s 984us/step - loss: 233.8245 - mae: 11.2203 - mse: 233.8245
MAE: 11.181077003479004   MSE: 232.3649139404297
Model saved successfully.
